# HRG (Heterogeneous Resource Gathering) 环境使用教程

本教程将介绍如何使用HRG环境进行多智能体强化学习实验。

## 目录
1. [环境简介](#环境简介)
2. [基础使用](#基础使用)
3. [CTDE兼容性](#ctde兼容性)
4. [可视化功能](#可视化功能)
5. [配置管理](#配置管理)
6. [性能测试](#性能测试)
7. [高级功能](#高级功能)

## 环境简介

HRG是一个异构资源收集环境，包含三种不同角色的智能体：
- **侦察兵(Scout)**：视野大，移动快，负责探索
- **工人(Worker)**：可以采集资源，负责生产
- **运输车(Transporter)**：容量大，负责物流

环境的目标是团队协作最大化资源收集效率。

# HRG环境详解：观测空间、动作空间与奖励机制

## 1. 观测空间（80维）详细解析

观测空间定义在 `env_hrg.py` 的 `_get_observation` 方法中，共80维，结构如下：

### 1.1 自身状态（10维）

| 维度 | 特征 | 描述 |
|------|------|------|
| 2 | 位置信息 | `position.x/grid_size`, `position.y/grid_size` - 归一化的坐标 |
| 3 | 角色类型 | one-hot编码 `[侦察兵, 工人, 运输车]` |
| 2 | 库存状态 | `gold/10.0`, `wood/10.0` - 归一化的资源数量 |
| 2 | 能量和冷却 | `energy/100.0`, `action_cooldown/2.0` - 能量和动作冷却时间 |
| 1 | 基地距离 | 到基地的曼哈顿距离归一化值 |

### 1.2 时间信息（1维）

- **剩余时间比例**：`1.0 - (current_step / max_steps)`
- 提供任务进度信息，帮助智能体规划长期策略

### 1.3 可见实体信息（最多50维）

最多观察10个实体，每个实体5维信息：

| 维度 | 特征 | 描述 |
|------|------|------|
| 2 | 相对位置 | `(pos.x - agent.x)/vision_range`, `(pos.y - agent.y)/vision_range` |
| 3 | 实体类型 | 
  - **其他智能体**：3维one-hot编码 `[侦察兵, 工人, 运输车]`
  - **资源**：2维one-hot编码 `[金矿, 木材]` + 1维资源数量 |

**关键特性**：
- 不同角色视野范围不同：侦察兵(5格) > 运输车(4格) > 工人(3格)
- 体现能力异构性，鼓励角色专业化

### 1.4 通信历史（10维）

最近3条消息的简化编码（基于发送者类型）：
- **侦察兵消息**：0.3
- **工人消息**：0.6  
- **运输车消息**：0.9

**设计哲学**：
- 模拟真实通信中的"信道质量"
- 避免直接传递完整语义，增加学习难度
- 鼓励智能体从有限信息中推断意图

## 2. 动作空间（8维）详细解析

动作空间定义在 `core.py` 中，包含8个离散动作：

### 2.1 移动动作（4个）

| 动作ID | 名称 | 描述 |
|--------|------|------|
| 0 | MOVE_NORTH | 向北移动 |
| 1 | MOVE_SOUTH | 向南移动 |
| 2 | MOVE_WEST | 向西移动 |
| 3 | MOVE_EAST | 向东移动 |

**物理约束**：
- 侦察兵：速度2.0（可连续移动两次）
- 工人：速度1.0
- 运输车：速度1.5（可移动1格+0.5概率移动第2格）

### 2.2 交互动作（4个）

| 动作ID | 名称 | 描述 |
|--------|------|------|
| 4 | GATHER | 采集当前位置的资源<br>• 工人专属动作，侦察兵无法采集<br>• 需要连续执行多个回合才能完成采集<br>• 采集时间：金矿4回合，木材2回合 |
| 5 | TRANSFER | 与相邻智能体转移资源<br>• 工人→运输车：转移采集到的资源<br>• 运输车→工人：通常为空操作<br>• 只有在相邻位置且有对应类型的智能体时才能执行 |
| 6 | DEPOSIT | 在基地存放资源<br>• 运输车专属动作，只有运输车可以存放资源<br>• 必须在基地位置(0,0)才能执行<br>• 获得资源价值的50%作为奖励 |
| 7 | WAIT | 等待/跳过回合 |

### 2.3 角色限制

不同角色有不同的动作限制：

| 动作 | 侦察兵 | 工人 | 运输车 |
|------|--------|------|--------|
| GATHER | ❌ | ✅ | ❌ |
| DEPOSIT | ❌ | ❌ | ✅ |
| TRANSFER | ✅ | ✅ | ✅ |
| MOVE | ✅ | ✅ | ✅ |

**设计优势**：
- 强制角色分化，避免所有智能体选择相同策略
- 模拟现实世界中的专业分工
- 促进团队协作需求

## 3. 奖励机制详细解析

奖励系统设计在 `env_hrg.py` 的 `_calculate_team_reward` 方法中，包含个人奖励和团队奖励。

### 3.1 个人动作奖励

#### 1. 移动奖励（_move_agent）
- **成功移动**：0.0（无奖励，仅消耗能量）
- **无效移动**：-0.1（撞墙或移动到无效位置）
- **能量消耗**：根据角色类型消耗不同能量
  - 侦察兵：0.05/步
  - 工人：0.02/步
  - 运输车：0.03/步

#### 2. 采集奖励（_gather_resource）
- **采集完成**：获得资源价值的10%
  - 金矿：10.0 × 0.1 = 1.0
  - 木材：2.0 × 0.1 = 0.2
- **采集过程中**：0.0（需要多回合完成）
- **能量消耗**：工人采集时消耗0.08能量

#### 3. 转移奖励（_transfer_resource）
- **成功转移**：获得转移资源价值的5%
  - 金矿：10.0 × 0.05 = 0.5
  - 木材：2.0 × 0.05 = 0.1
- **能量消耗**：转移时消耗0.1能量

#### 4. 存放奖励（_deposit_resources）
- **成功存放**：获得资源价值的50%
  - 金矿：10.0 × 0.5 = 5.0
  - 木材：2.0 × 0.5 = 1.0
- 这是最大的单次奖励，鼓励资源成功运回基地

### 3.2 团队奖励（_calculate_team_reward）

| 奖励类型 | 值 | 描述 |
|---------|-----|------|
| 时间惩罚 | -0.01/步 | 鼓励快速完成任务 |
| 资源多样性奖励 | +0.1 (金矿), +0.05 (木材) | 第一次存放特定资源时给予额外奖励 |
| 协作奖励 | 动态计算 | 基于团队整体效率的附加奖励 |

### 3.3 角色能力差异

| 角色 | 优势 | 限制 | 能量消耗 |
|------|------|------|----------|
| **侦察兵** | 视野范围5，移动速度2.0 | 无法采集，携带容量0 | 移动0.05 |
| **工人** | 可采集，携带容量2 | 视野范围3，移动速度1.0 | 移动0.02，采集0.08 |
| **运输车** | 可存放资源，携带容量5，移动速度1.5 | 无法采集，视野范围4 | 移动0.03，转移0.1 |

## 4. 奖励设计理念

### 4.1 核心原则

1. **鼓励协作**
   - 转移奖励(0.5) < 存放奖励(5.0)，形成"采集→转移→存放"的价值链
   - 团队共享最终奖励，强化集体目标

2. **效率导向**
   - 每步-0.01的时间惩罚，促使智能体快速行动
   - 能量消耗机制防止无限循环行为

3. **角色专精**
   - 不同角色有不同的最优策略
   - 侦察兵应专注于探索和通信
   - 工人应专注于资源采集
   - 运输车应专注于物流调度

4. **目标明确**
   - 最终目标是收集并存放资源到基地
   - 奖励信号清晰指向核心任务

### 4.2 多层次奖励结构

```
+-----------------------+
|      最终目标         |
|  资源存放至基地       |
+-----------------------+
           |
           v
+-----------------------+
|     中间目标          |
|  资源采集与转移       |
+-----------------------+
           |
           v
+-----------------------+
|     基础行为          |
|  探索、移动、避障     |
+-----------------------+
```

## 5. 总结

HRG环境是一个典型的异构多智能体协作环境，具有以下特点：

- **80维观测空间**：提供了丰富的环境信息，包括自身状态、时间信息、可见实体和通信历史
- **8维动作空间**：支持复杂的决策，包含移动、采集、转移和存放等交互动作
- **精心设计的奖励机制**：通过多层次奖励引导智能体学习高效协作策略

**环境优势**：
1. **角色分化需求高**：强制不同角色承担不同职责
2. **通信必要性强**：侦察兵发现资源后必须通知工人
3. **物流链条完整**：从探索到采集再到运输形成完整闭环
4. **挑战性适中**：既不过于简单也不过于复杂，适合算法测试

这个环境为测试多智能体协作算法提供了一个理想的平台，特别是对于验证RACGA等强调角色动态性和双通道通信的方法。

In [1]:
# 导入必要的库
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# 添加环境路径
import sys
sys.path.append("..")  # 将上级目录添加到系统路径

# 导入HRG环境
from Env.HRG import create_hrg_env, create_hrg_ctde_env
from Env.HRG.config import get_config_by_name
from Env.HRG.renderer import MatplotlibRenderer

print("HRG环境导入成功！")

HRG环境导入成功！


## 基础使用

In [2]:
# 创建基础HRG环境
env = create_hrg_env(difficulty="normal", render_mode="")

print(f"环境信息:")
print(f"- 智能体数量: {env.n_agents}")
print(f"- 智能体ID: {env.agent_ids}")
print(f"- 网格大小: {env.config.grid_size}x{env.config.grid_size}")
print(f"- 最大步数: {env.config.max_steps}")
print(f"- 观察空间维度: {list(env.obs_dims.values())[0]}")
print(f"- 动作空间维度: {list(env.act_dims.values())[0]}")

环境信息:
- 智能体数量: 6
- 智能体ID: ['scout_0', 'scout_1', 'transporter_0', 'worker_0', 'worker_1', 'worker_2']
- 网格大小: 10x10
- 最大步数: 200
- 观察空间维度: 80
- 动作空间维度: 8


In [3]:
# 重置环境
observations = env.reset()

print("初始观察信息:")
for agent_id, obs in observations.items():
    print(f"- {agent_id}: 观察形状={obs.shape}, 数据类型={obs.dtype}")
    print(f"  最小值={obs.min():.3f}, 最大值={obs.max():.3f}")
    break  # 只显示第一个智能体的信息

初始观察信息:
- scout_0: 观察形状=(80,), 数据类型=float32
  最小值=-0.200, 最大值=1.000


In [4]:
# 执行一个简单的回合
total_rewards = {agent_id: 0 for agent_id in env.agent_ids}
step_count = 0

print("开始执行回合...")

while step_count < 50:  # 限制步数用于演示
    # 随机选择动作
    actions = {}
    for agent_id in env.agent_ids:
        # 获取可用动作
        avail_actions = env.get_avail_actions(agent_id)
        action = np.random.choice(avail_actions)
        actions[agent_id] = action
    
    # 执行动作
    observations, rewards, dones, infos = env.step(actions)
    
    # 累积奖励
    for agent_id, reward in rewards.items():
        total_rewards[agent_id] += reward
    
    step_count += 1
    
    # 打印进度
    if step_count % 10 == 0:
        avg_reward = np.mean(list(rewards.values()))
        print(f"步数 {step_count}: 平均奖励 = {avg_reward:.3f}")
    
    # 检查是否结束
    if any(dones.values()):
        print(f"回合在步数 {step_count} 结束")
        break

print("\n回合结果:")
for agent_id, total_reward in total_rewards.items():
    print(f"- {agent_id}: 总奖励 = {total_reward:.3f}")
print(f"- 团队平均奖励: {np.mean(list(total_rewards.values())):.3f}")
print(f"- 环境总得分: {env.game_state.total_score:.1f}")

开始执行回合...
步数 10: 平均奖励 = -0.010
步数 20: 平均奖励 = -0.060
步数 30: 平均奖励 = -0.010
步数 40: 平均奖励 = -0.027
步数 50: 平均奖励 = -0.060

回合结果:
- scout_0: 总奖励 = -1.700
- scout_1: 总奖励 = -3.000
- transporter_0: 总奖励 = -1.200
- worker_0: 总奖励 = -1.900
- worker_1: 总奖励 = -1.100
- worker_2: 总奖励 = -1.000
- 团队平均奖励: -1.650
- 环境总得分: 0.0


In [5]:
# 关闭环境
env.close()
print("环境已关闭")

环境已关闭


## CTDE兼容性

HRG环境提供了CTDE（集中式训练分布式执行）包装器，支持QMIX、VDN等算法。

In [6]:
# 测试不同类型的全局状态表示
global_state_types = ["concat", "mean", "max", "attention"]

for state_type in global_state_types:
    print(f"\n测试全局状态类型: {state_type}")
    
    # 创建CTDE环境
    ctde_env = create_hrg_ctde_env(
        "normal_ctde",
        global_state_type=state_type
    )
    
    # 重置环境
    observations = ctde_env.reset()
    
    # 获取全局状态
    global_state = ctde_env.get_global_state()
    
    print(f"  - 全局状态维度: {global_state.shape}")
    print(f"  - 非零元素数量: {np.count_nonzero(global_state)}")
    print(f"  - 数据范围: [{global_state.min():.3f}, {global_state.max():.3f}]")
    
    # 执行一步并检查info中的全局状态
    actions = {agent_id: 7 for agent_id in ctde_env.agent_ids}
    obs, rewards, dones, infos = ctde_env.step(actions)
    
    if 'global_state' in infos:
        info_state = infos['global_state']
        print(f"  - Info中全局状态形状: {info_state.shape}")
        print(f"  - 状态一致性: {np.allclose(global_state, info_state)}")
    
    ctde_env.close()


测试全局状态类型: concat
  - 全局状态维度: (480,)
  - 非零元素数量: 72
  - 数据范围: [-0.200, 1.000]
  - Info中全局状态形状: (480,)
  - 状态一致性: False

测试全局状态类型: mean
  - 全局状态维度: (80,)
  - 非零元素数量: 14
  - 数据范围: [-0.247, 1.000]
  - Info中全局状态形状: (80,)
  - 状态一致性: False

测试全局状态类型: max
  - 全局状态维度: (80,)
  - 非零元素数量: 14
  - 数据范围: [0.000, 1.000]
  - Info中全局状态形状: (80,)
  - 状态一致性: False

测试全局状态类型: attention
  - 全局状态维度: (86,)
  - 非零元素数量: 21
  - 数据范围: [-0.171, 1.000]
  - Info中全局状态形状: (86,)
  - 状态一致性: False


In [7]:
# 获取CTDE环境信息
ctde_env = create_hrg_ctde_env("normal_ctde")
env_info = ctde_env.get_env_info()

print("CTDE环境信息:")
for key, value in env_info.items():
    print(f"- {key}: {value}")

ctde_env.close()

CTDE环境信息:
- env_name: HRG-CTDE
- n_agents: 6
- agent_ids: ['scout_0', 'scout_1', 'transporter_0', 'worker_0', 'worker_1', 'worker_2']
- obs_dims: {'scout_0': 80, 'scout_1': 80, 'transporter_0': 80, 'worker_0': 80, 'worker_1': 80, 'worker_2': 80}
- act_dims: {'scout_0': 8, 'scout_1': 8, 'transporter_0': 8, 'worker_0': 8, 'worker_1': 8, 'worker_2': 8}
- global_state_dim: 480
- episode_limit: 200
- global_state_type: concat


## 配置管理

HRG环境提供了丰富的配置选项，支持不同的实验设置。

In [12]:
# 测试不同难度配置
difficulties = ["easy", "normal", "hard"]

for difficulty in difficulties:
    print(f"\n难度配置: {difficulty}")
    
    env = create_hrg_env(difficulty=difficulty)
    
    print(f"  - 网格大小: {env.config.grid_size}x{env.config.grid_size}")
    print(f"  - 最大步数: {env.config.max_steps}")
    print(f"  - 障碍物数量: {env.config.num_obstacles}")
    print(f"  - 金矿数量: {env.config.num_gold}")
    print(f"  - 木材数量: {env.config.num_wood}")
    
    env.close()


难度配置: easy
  - 网格大小: 8x8
  - 最大步数: 300
  - 障碍物数量: 0
  - 金矿数量: 2
  - 木材数量: 15

难度配置: normal
  - 网格大小: 10x10
  - 最大步数: 200
  - 障碍物数量: 10
  - 金矿数量: 3
  - 木材数量: 10

难度配置: hard
  - 网格大小: 12x12
  - 最大步数: 150
  - 障碍物数量: 20
  - 金矿数量: 4
  - 木材数量: 8


In [13]:
# 测试专用配置
special_configs = ["communication", "coordination", "exploration", "role_test"]

for config_name in special_configs:
    print(f"\n专用配置: {config_name}")
    
    try:
        config = get_config_by_name(config_name)
        env = create_hrg_env(difficulty="normal")
        
        # 更新配置
        env.config.grid_size = config.grid_size
        env.config.max_steps = config.max_steps
        env.config.num_obstacles = config.num_obstacles
        
        print(f"  - 配置描述: {config_name}")
        print(f"  - 网格: {config.grid_size}x{config.grid_size}")
        print(f"  - 步数: {config.max_steps}")
        print(f"  - 障碍物: {config.num_obstacles}")
        
        env.close()
        
    except Exception as e:
        print(f"  - 错误: {e}")


专用配置: communication
  - 配置描述: communication
  - 网格: 10x10
  - 步数: 250
  - 障碍物: 15

专用配置: coordination
  - 配置描述: coordination
  - 网格: 10x10
  - 步数: 200
  - 障碍物: 8

专用配置: exploration
  - 配置描述: exploration
  - 网格: 15x15
  - 步数: 400
  - 障碍物: 25

专用配置: role_test
  - 配置描述: role_test
  - 网格: 10x10
  - 步数: 200
  - 障碍物: 12


In [14]:
# 自定义配置示例
from Env.HRG.env_hrg import HRGConfig

# 创建自定义配置
custom_config = HRGConfig(
    grid_size=8,
    max_steps=100,
    num_obstacles=5,
    num_gold=2,
    num_wood=8,
    render_mode="rgb_array"
)

print("自定义配置:")
print(f"- 网格大小: {custom_config.grid_size}")
print(f"- 最大步数: {custom_config.max_steps}")
print(f"- 障碍物: {custom_config.num_obstacles}")
print(f"- 金矿: {custom_config.num_gold}")
print(f"- 木材: {custom_config.num_wood}")

# 创建环境
custom_env = create_hrg_env(difficulty="normal")
custom_env.config = custom_config

print(f"\n自定义环境创建成功！")
custom_env.close()

自定义配置:
- 网格大小: 8
- 最大步数: 100
- 障碍物: 5
- 金矿: 2
- 木材: 8

自定义环境创建成功！


## 性能测试

评估环境的执行性能，确保适合强化学习训练。

In [16]:
import time

# 性能测试
def benchmark_environment(num_steps=100):
    env = create_hrg_env(difficulty="normal", render_mode="")
    observations = env.reset()
    
    start_time = time.time()
    
    total_rewards = []
    step_times = []
    
    for step in range(num_steps):
        step_start = time.time()
        
        # 随机动作
        actions = {agent_id: np.random.choice(env.get_avail_actions(agent_id)) 
                  for agent_id in env.agent_ids}
        
        # 执行步骤
        observations, rewards, dones, infos = env.step(actions)
        
        step_end = time.time()
        step_times.append(step_end - step_start)
        total_rewards.append(np.mean(list(rewards.values())))
        
        # 如果回合结束，重置
        if any(dones.values()):
            observations = env.reset()
    
    total_time = time.time() - start_time
    
    env.close()
    
    return {
        'total_time': total_time,
        'avg_step_time': np.mean(step_times),
        'steps_per_second': num_steps / total_time,
        'avg_reward': np.mean(total_rewards),
        'max_step_time': np.max(step_times),
        'min_step_time': np.min(step_times)
    }

# 运行基准测试
results = benchmark_environment(200)

print("性能测试结果:")
print(f"- 总时间: {results['total_time']:.3f}秒")
print(f"- 平均每步时间: {results['avg_step_time']:.4f}秒")
print(f"- 每秒步数: {results['steps_per_second']:.1f}")
print(f"- 最大步时间: {results['max_step_time']:.4f}秒")
print(f"- 最小步时间: {results['min_step_time']:.4f}秒")
print(f"- 平均奖励: {results['avg_reward']:.4f}")

# 性能评估
if results['avg_step_time'] < 0.01:
    print("\n🚀 性能优秀: 适合大规模训练")
elif results['avg_step_time'] < 0.05:
    print("\n✅ 性能良好: 适合常规训练")
elif results['avg_step_time'] < 0.1:
    print("\n⚠️  性能一般: 可以用于训练，但可能较慢")
else:
    print("\n❌ 性能较慢: 建议优化后再进行大规模训练")

性能测试结果:
- 总时间: 0.172秒
- 平均每步时间: 0.0008秒
- 每秒步数: 1160.7
- 最大步时间: 0.0013秒
- 最小步时间: 0.0007秒
- 平均奖励: -0.0285

🚀 性能优秀: 适合大规模训练


## 高级功能

演示环境的高级功能，包括动作掩码、详细观察分析等。

In [17]:
# 动作掩码演示
env = create_hrg_env(difficulty="normal", render_mode="")
observations = env.reset()

print("动作掩码演示:")
for agent_id in env.agent_ids:
    avail_actions = env.get_avail_actions(agent_id)
    print(f"\n智能体 {agent_id}:")
    print(f"  - 可用动作数量: {len(avail_actions)}")
    print(f"  - 可用动作: {avail_actions}")
    
    # 动作名称映射
    action_names = {
        0: "上移动", 1: "下移动", 2: "左移动", 3: "右移动",
        4: "采集", 5: "传输", 6: "存放", 7: "等待"
    }
    
    print(f"  - 可用动作名称: {[action_names[a] for a in avail_actions]}")

env.close()

动作掩码演示:

智能体 scout_0:
  - 可用动作数量: 6
  - 可用动作: [0, 1, 2, 3, 5, 7]
  - 可用动作名称: ['上移动', '下移动', '左移动', '右移动', '传输', '等待']

智能体 scout_1:
  - 可用动作数量: 6
  - 可用动作: [0, 1, 2, 3, 5, 7]
  - 可用动作名称: ['上移动', '下移动', '左移动', '右移动', '传输', '等待']

智能体 transporter_0:
  - 可用动作数量: 8
  - 可用动作: [0, 1, 2, 3, 4, 5, 6, 7]
  - 可用动作名称: ['上移动', '下移动', '左移动', '右移动', '采集', '传输', '存放', '等待']

智能体 worker_0:
  - 可用动作数量: 7
  - 可用动作: [0, 1, 2, 3, 4, 5, 7]
  - 可用动作名称: ['上移动', '下移动', '左移动', '右移动', '采集', '传输', '等待']

智能体 worker_1:
  - 可用动作数量: 7
  - 可用动作: [0, 1, 2, 3, 4, 5, 7]
  - 可用动作名称: ['上移动', '下移动', '左移动', '右移动', '采集', '传输', '等待']

智能体 worker_2:
  - 可用动作数量: 7
  - 可用动作: [0, 1, 2, 3, 4, 5, 7]
  - 可用动作名称: ['上移动', '下移动', '左移动', '右移动', '采集', '传输', '等待']

智能体 scout_0:
  - 可用动作数量: 6
  - 可用动作: [0, 1, 2, 3, 5, 7]
  - 可用动作名称: ['上移动', '下移动', '左移动', '右移动', '传输', '等待']

智能体 scout_1:
  - 可用动作数量: 6
  - 可用动作: [0, 1, 2, 3, 5, 7]
  - 可用动作名称: ['上移动', '下移动', '左移动', '右移动', '传输', '等待']

智能体 worker_0:
  - 可用动作数量: 7
  - 可用动作: [0, 1, 2, 3, 4, 5, 

In [18]:
# 观察空间详细分析
env = create_hrg_env(difficulty="normal")
observations = env.reset()

print("观察空间详细分析:")

# 分析一个智能体的观察
sample_agent_id = env.agent_ids[0]
obs = observations[sample_agent_id]

print(f"\n智能体 {sample_agent_id} 的观察分析:")
print(f"- 观察维度: {obs.shape}")
print(f"- 数据类型: {obs.dtype}")
print(f"- 最小值: {obs.min():.4f}")
print(f"- 最大值: {obs.max():.4f}")
print(f"- 均值: {obs.mean():.4f}")
print(f"- 标准差: {obs.std():.4f}")
print(f"- 零值比例: {np.mean(obs == 0):.2%}")

# 按观察组成部分分析
print("\n观察组成部分分析:")
print(f"- 自身状态 (0-10): min={obs[:10].min():.3f}, max={obs[:10].max():.3f}")
print(f"- 视野内实体 (10-60): min={obs[10:60].min():.3f}, max={obs[10:60].max():.3f}")
print(f"- 消息信息 (60-70): min={obs[60:70].min():.3f}, max={obs[60:70].max():.3f}")
print(f"- 其他信息 (70-80): min={obs[70:80].min():.3f}, max={obs[70:80].max():.3f}")

env.close()

观察空间详细分析:

智能体 scout_0 的观察分析:
- 观察维度: (80,)
- 数据类型: float32
- 最小值: -0.4000
- 最大值: 1.0000
- 均值: 0.0812
- 标准差: 0.2941
- 零值比例: 83.75%

观察组成部分分析:
- 自身状态 (0-10): min=0.000, max=1.000
- 视野内实体 (10-60): min=-0.400, max=1.000
- 消息信息 (60-70): min=0.000, max=0.000
- 其他信息 (70-80): min=0.000, max=0.000


In [19]:
# 智能体角色能力分析
env = create_hrg_env(difficulty="normal")
observations = env.reset()

print("智能体角色能力分析:")

from Env.HRG.core import AgentType

role_stats = {
    AgentType.SCOUT: [],
    AgentType.WORKER: [],
    AgentType.TRANSPORTER: []
}

# 收集几个回合的数据
for episode in range(3):
    observations = env.reset()
    
    for agent in env.agents.values():
        role_stats[agent.type].append({
            'vision_range': agent.config.vision_range,
            'move_speed': agent.config.move_speed,
            'carry_capacity': agent.config.carry_capacity,
            'energy': agent.energy
        })

# 统计分析
for agent_type, stats in role_stats.items():
    if stats:
        print(f"\n{agent_type.name}:")
        print(f"  - 数量: {len(stats)}")
        print(f"  - 视野范围: {stats[0]['vision_range']}")
        print(f"  - 移动速度: {stats[0]['move_speed']}")
        print(f"  - 负重能力: {stats[0]['carry_capacity']}")
        print(f"  - 初始能量: {stats[0]['energy']}")

env.close()

智能体角色能力分析:

SCOUT:
  - 数量: 6
  - 视野范围: 5
  - 移动速度: 2.0
  - 负重能力: 0
  - 初始能量: 100.0

WORKER:
  - 数量: 9
  - 视野范围: 3
  - 移动速度: 1.0
  - 负重能力: 2
  - 初始能量: 100.0

TRANSPORTER:
  - 数量: 3
  - 视野范围: 4
  - 移动速度: 1.5
  - 负重能力: 5
  - 初始能量: 100.0


In [20]:
# 奖励系统分析
env = create_hrg_env(difficulty="normal")

print("奖励系统分析:")

# 执行多个步骤收集奖励数据
all_rewards = {agent_id: [] for agent_id in env.agent_ids}
episode_rewards = []

for episode in range(5):
    observations = env.reset()
    episode_total = 0
    
    for step in range(50):
        actions = {agent_id: np.random.choice(env.get_avail_actions(agent_id)) 
                  for agent_id in env.agent_ids}
        observations, rewards, dones, infos = env.step(actions)
        
        for agent_id, reward in rewards.items():
            all_rewards[agent_id].append(reward)
            episode_total += reward
        
        if any(dones.values()):
            break
    
    episode_rewards.append(episode_total)
    print(f"回合 {episode + 1}: 总奖励 = {episode_total:.3f}, 得分 = {env.game_state.total_score:.1f}")

print("\n奖励统计分析:")
for agent_id, rewards in all_rewards.items():
    if rewards:
        print(f"\n{agent_id}:")
        print(f"  - 平均奖励: {np.mean(rewards):.4f}")
        print(f"  - 标准差: {np.std(rewards):.4f}")
        print(f"  - 最大奖励: {np.max(rewards):.4f}")
        print(f"  - 最小奖励: {np.min(rewards):.4f}")
        print(f"  - 正奖励比例: {np.mean(np.array(rewards) > 0):.2%}")

print(f"\n团队平均回合奖励: {np.mean(episode_rewards):.3f} ± {np.std(episode_rewards):.3f}")

env.close()

奖励系统分析:
回合 1: 总奖励 = -6.900, 得分 = 0.0
回合 2: 总奖励 = -8.200, 得分 = 0.0
回合 3: 总奖励 = -6.000, 得分 = 0.0
回合 4: 总奖励 = -7.500, 得分 = 0.0
回合 5: 总奖励 = -6.200, 得分 = 0.0

奖励统计分析:

scout_0:
  - 平均奖励: -0.0284
  - 标准差: 0.0387
  - 最大奖励: -0.0100
  - 最小奖励: -0.1100
  - 正奖励比例: 0.00%

scout_1:
  - 平均奖励: -0.0312
  - 标准差: 0.0409
  - 最大奖励: -0.0100
  - 最小奖励: -0.1100
  - 正奖励比例: 0.00%

transporter_0:
  - 平均奖励: -0.0260
  - 标准差: 0.0367
  - 最大奖励: -0.0100
  - 最小奖励: -0.1100
  - 正奖励比例: 0.00%

worker_0:
  - 平均奖励: -0.0144
  - 标准差: 0.0257
  - 最大奖励: 0.1900
  - 最小奖励: -0.1100
  - 正奖励比例: 0.40%

worker_1:
  - 平均奖励: -0.0204
  - 标准差: 0.0305
  - 最大奖励: -0.0100
  - 最小奖励: -0.1100
  - 正奖励比例: 0.00%

worker_2:
  - 平均奖励: -0.0188
  - 标准差: 0.0323
  - 最大奖励: 0.1900
  - 最小奖励: -0.1100
  - 正奖励比例: 0.40%

团队平均回合奖励: -6.960 ± 0.816


In [21]:
env.close()

## 总结

本教程介绍了HRG环境的主要功能和使用方法：

1. **基础环境创建和使用**：展示了如何创建环境、重置、执行步骤
2. **CTDE兼容性**：演示了集中式训练分布式执行的支持
3. **可视化功能**：展示了静态和动态可视化能力
4. **配置管理**：介绍了不同难度和专用配置
5. **性能测试**：评估了环境执行效率
6. **高级功能**：演示了动作掩码、观察分析等功能
7. **完整示例**：提供了简单智能体的实现

### 关键特点：
- ✅ **异构智能体**：三种不同角色，各具特色
- ✅ **部分可观测**：不同视野范围，信息不对称
- ✅ **协作需求**：需要智能体团队协作才能高效完成任务
- ✅ **CTDE兼容**：支持主流多智能体强化学习算法
- ✅ **高性能**：单步执行时间<0.1秒，适合大规模训练
- ✅ **可视化**：丰富的可视化功能便于分析和调试
- ✅ **可配置**：多种预设和自定义配置支持

HRG环境现在可以用于多智能体强化学习的研究和实验了！